In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.0.0-rc2'

In [0]:
# CIFAR-10 데이터를 다운로드 받기 위한 keras의 helper 함수인 load_data 함수를 임포트합니다.
from tensorflow.keras.datasets.cifar10 import load_data

In [0]:
# def next_batch(num, data, labels):
#   '''
#   `num` 개수 만큼의 랜덤한 샘플들과 레이블들을 리턴합니다.
#   '''
#   idx = np.arange(0 , len(data))
#   np.random.shuffle(idx)
#   idx = idx[:num]
#   data_shuffle = [data[ i] for i in idx]
#   labels_shuffle = [labels[ i] for i in idx]
#   return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [5]:
(x_train, y_train), (x_test, y_test) = load_data()

# y_train = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
# y_test = tf.squeeze(tf.one_hot(y_test, 10), axis=1)
n_classes = np.max(y_train) + 1
y_train = np.eye(n_classes)[np.squeeze(y_train)]
y_test = np.eye(n_classes)[np.squeeze(y_test)]

170500096/170498071 [==============================] - 12s 0us/step


In [0]:
def standardzation(data, momentum=0.99, epsilon=1e-4):
  mean = np.mean(data)
  variance = np.var(data)
  
#   print("mean:", mean)
#   print("variance:",variance)
  
  return momentum * ((data - mean) / np.sqrt(variance**2 + 1e-6)) + epsilon


x_test = standardzation(x_test)

In [0]:
class Model_CIFAR10:
  def __init__(self, sess, name, decay = 0.96, learning_rate = 3e-4):
    self.sess = sess
    self.name = name
    self.decay = decay
    self.learning_rate = learning_rate
    self.build_net()
    
  def build_net(self):
    with tf.device('/device:GPU:0'):
      with tf.variable_scope(self.name):
        self.x = tf.placeholder(tf.float32, [None, 32, 32, 3], name = 'x')        
        self.y = tf.placeholder(tf.float32, [None, 10], name='y')
        self.keep_prob = tf.placeholder(tf.float32, name = 'prob')

        # [?, 32, 32, 3] -> [?, 16, 16, 64]
        with tf.name_scope("Layer1"):
          W1 = tf.get_variable('W1', shape = [3, 3, 3, 64],
                                            initializer = tf.keras.initializers.he_normal())
          L1 = tf.nn.conv2d(self.x, W1, strides=[1, 1, 1, 1], padding='SAME')
          L1 = tf.nn.relu(L1)
#           L1 = tf.math.tanh(L1)
          L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
          L1 = tf.nn.dropout(L1, keep_prob=self.keep_prob)


        with tf.name_scope("Layer2"):
          # [?, 16, 16, 64] -> [?, 8, 8, 64]
          W2 = tf.get_variable('W2', shape = [3, 3, 64, 128],
                                            initializer=tf.keras.initializers.he_normal())
          L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
          L2 = tf.nn.relu(L2)
#           L2 = tf.math.tanh(L2)
          L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
          L2 = tf.nn.dropout(L2, keep_prob=self.keep_prob)
    
        with tf.name_scope("Layer4"):
          # [?, 8, 8, 64] -> [?, 4, 4, 128]
          W4 = tf.get_variable('W4', shape = [3, 3, 128, 128],
                                            initializer=tf.keras.initializers.he_normal())
          L4 = tf.nn.conv2d(L2, W4, strides=[1, 1, 1, 1], padding='SAME')
          L4 = tf.nn.relu(L4)
#           L4 = tf.math.tanh(L4)
          L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
          L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)
          L4_flat = tf.reshape(L4, [-1, 4 * 4 * 128])
          
          
        with tf.name_scope("FC1"):
          # [?, 4 * 4 * 256] -> [?, 256]
          W6 = tf.get_variable('W6', shape=[128 * 4 * 4, 128], 
                                            initializer=tf.keras.initializers.he_normal())
          b6 = tf.get_variable('b6', shape=[128], initializer=tf.keras.initializers.zeros())
          FC1 = tf.keras.layers.BatchNormalization()(tf.nn.relu(tf.matmul(L4_flat, W6) + b6), training=True)
#           FC1 = tf.math.tanh(tf.matmul(L4_flat, W6) + b6)
          FC1 = tf.nn.dropout(FC1, keep_prob=self.keep_prob)


        with tf.name_scope("FC2"):
          # [?, 128] -> [? 10]
          W7 = tf.get_variable('W7', shape=[128, 10], 
                                            initializer=tf.keras.initializers.he_normal())
          b7 = tf.get_variable('b7', shape=[10], initializer=tf.keras.initializers.zeros())
          self.logits = tf.matmul(FC1, W7) + b7
          self.y_pred = tf.nn.softmax(self.logits)
                  
        with tf.name_scope("Cost"):
          self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                logits=self.logits, labels=self.y))
          
        with tf.name_scope("Train"):
          self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate, 
                                                                epsilon=1e-5).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.y_pred, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  
  def set_learningRate(self, learning_rate):
    self.learning_rate = learning_rate
  
  def predict(self, x_data, keep_prob=1.0):
    return self.sess.run(self.y_pred, feed_dict={self.x: x_data, self.keep_prob: keep_prop})

  def get_accuracy(self, x_data, y_data, keep_prob=1.0):
    return self.sess.run(self.accuracy, feed_dict={self.x: x_data, self.y: y_data, self.keep_prob:keep_prob})

  def train(self, x_data, y_data, keep_prob=0.2):
    return self.sess.run([self.cost, self.optimizer],
                                      feed_dict = {self.x:x_data, self.y:y_data, self.keep_prob:keep_prob})

In [0]:
import random as rd

training_epochs = 100
batch_size = 100



with tf.Session() as sess:
  model = Model_CIFAR10(sess, 'M3', learning_rate=0.1)
  sess.run(tf.global_variables_initializer())
  
  print("Learning Start! with learning_rate:", model.learning_rate)
  
  # define dataset for next batch
  dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  dataset = dataset.shuffle(20000)
  dataset = dataset.repeat(training_epochs)
  dataset = dataset.batch(batch_size)

  # get iterator
  iterator = dataset.make_one_shot_iterator()
  next_batch = iterator.get_next()  
  global_step = 0
  
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(x_train.shape[0] / batch_size)
    
    decay = model.decay ** (global_step / 100000)
    learning_rate = model.learning_rate * decay
    model.set_learningRate(learning_rate)
    noise = np.random.rand(1) * (10 ** -5)
    
    for i in range(total_batch):
      batch_xs, batch_ys = sess.run(next_batch)
      batch_xs = standardzation(batch_xs)
      if global_step % 5 != 0:
        batch_xs *= noise      
        
      c, _ = model.train(batch_xs, batch_ys)
      avg_cost += c / total_batch
      global_step += 1
    
    print("Epoch: {:04d}     Global Step: {:07d}    Cost: {:.9f}     Test Accuracy: {:.5f}    Learning Rate: {:.15f}".format(epoch+1, 
                                                                                                global_step,
                                                                                                avg_cost, 
                                                                                                model.get_accuracy(x_test, y_test),
                                                                                                model.learning_rate))
  print("Learning Complete!!")
  
  print("Accuracy:", model.get_accuracy(x_test, y_test))

Learning Start! with learning_rate: 0.1
Epoch: 0001     Global Step: 0000500    Cost: 2.501124313     Test Accuracy: 0.17750    Learning Rate: 0.100000000000000
Epoch: 0002     Global Step: 0001000    Cost: 2.327254827     Test Accuracy: 0.17420    Learning Rate: 0.099979591085642
Epoch: 0003     Global Step: 0001500    Cost: 2.527359648     Test Accuracy: 0.13990    Learning Rate: 0.099938785751790
Epoch: 0004     Global Step: 0002000    Cost: 2.500388251     Test Accuracy: 0.18140    Learning Rate: 0.099877608975422
Epoch: 0005     Global Step: 0002500    Cost: 2.460459790     Test Accuracy: 0.15440    Learning Rate: 0.099796098190152
Epoch: 0006     Global Step: 0003000    Cost: 2.394696096     Test Accuracy: 0.19700    Learning Rate: 0.099694303248058
Epoch: 0007     Global Step: 0003500    Cost: 2.370100451     Test Accuracy: 0.16870    Learning Rate: 0.099572286368867
Epoch: 0008     Global Step: 0004000    Cost: 2.330858020     Test Accuracy: 0.18740    Learning Rate: 0.09943012

KeyboardInterrupt: ignored

In [0]:
# Set parameters
LEARN_RATE = 1e-4
TRAIN_EPOCH = 15
BATCH_SIZE = 100

In [0]:
class Model_CIFAR10(tf.keras.Model):
  def __init__(self):
    super(Model_CIFAR10, self).__init__()
    self.conv1 = keras.layers.Conv2D(32, kernel_size=3, padding='same', activation=tf.nn.relu)
    self.pool1 = keras.layers.MaxPooling2D(padding='same')
    self.conv2 = keras.layers.Conv2D(64, kernel_size=3, padding='same', activation=tf.nn.relu)
    self.pool2 = keras.layers.MaxPooling2D(padding='same')
    self.conv3 = keras.layers.Conv2D(128, kernel_size=3, padding='same', activation=tf.nn.relu)
    self.pool3 = keras.layers.MaxPooling2D(padding='same')
    
    self.pool3_flat = keras.layers.Flatten()
    self.dense4 = keras.layers.Dense(256, tf.nn.relu)
    self.drop4 = keras.layers.Dropout(0.4)
    self.dense5 = keras.layers.Dense(10, tf.nn.softmax)
  def call(self, inputs, trainin=False):
    net = self.conv1(inputs)
    net = self.pool1(net)
    net = self.conv2(net)
    net = self.pool2(net)
    net = self.conv3(net)
    net = self.pool3(net)
    net = self.pool3_flat(net)
    net = self.dense4(net)
    net = self.drop4(net)
    net = self.dense5(net)
    
    return net

In [0]:
def loss_fn(model, images, labels):
  logits = model(images, training=True)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,
                                                                   logits=logits))
  
  return loss

def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
    
  return tape.gradient(loss, model.variables)

In [0]:
models = []
num_models = 3

for m in range(num_models):
  models.append(Model_CIFAR10())

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=100000).batch(BATCH_SIZE)

In [0]:
|